# Модули

## Введение

Среднестатистическая программа, реализованная на языке Python, состоит из нескольких исходных файлов с расширением *.py*. Каждый исходный файл является **модулем**. Обычно модули независимы друг от друга и могут быть использованы в вножестве различных программ в случаях необходимости. Модули, которые являются связанными друг с другом как в кодовой форме, так и в форме некоторой бизнес логики, часто хранятся в виде **пакетов** (англ. *package*) - древовидных иерархических модульных структур. Пример подобных структур вы могли видеть в [первой домашней работе](../../homeworks/hw1/description.md):

```console
lsm_project
├───event_logger
│   ├───__init__.py
│   └───event_logger.py
├───lsm
│   ├───__init__.py
│   ├───enumerations.py
│   ├───functions.py
│   └───models.py
└───visualization.py
```

Помимо обычных модулей, реализованных на языке Python, также существуют **модули расширения** (англ. *extension modules*) - модули написанные на других языках программирования, например, на C для CPython, или на Java для Jython. Эти модули также могут быть использованы в вашем Python коде, и обычно пишутся с целью оптимизации тех частей кода, которые отрабатывают недостаточно быстро, или для расширения функционала самого Python и доступа к более нискому уровню абстракции.

В этой лекции речь пойдет только о модулях, написанных на языке Python.

## Общие сведения

Функции в Python - это отдельные объекты. Классы в Python - это отдельные объекты. Модули в Python, как это ни странно, также являются отдельными объектами. Корректно будет сказать, что модули в Python - это самостоятельные объекты, хранящие независимо именованные атрибуты. Код, описывающий модуль с именем `modul_name` обычно хранится в файле `module_name.py`.

Поскольку модули - это объекты, вы можете работать с модулями так же, как и с другими объектами. Таким образом, модуль может быть передан в качестве аргумента при вызове функции:

In [10]:
from typing import Any


def print_object_info(obj: Any) -> None:
    print(
        f'object type: {type(obj).__name__}',
        f'object value: {obj}',
        sep='\n',
        end='\n\n'
    )

In [11]:
import typing


my_list = list(range(5))

print_object_info(my_list)
print_object_info(typing)

object type: list
object value: [0, 1, 2, 3, 4]

object type: module
object value: <module 'typing' from 'c:\\Users\\Michail\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\typing.py'>



Модуль может быть возвращен в качестве результата выполнения функции и привязан к переменной:

In [1]:
from enum import Enum


class Devices(Enum):
    CPU = 'cpu'
    GPU = 'gpu'

In [2]:
from types import ModuleType
from typing import Union


def load_computition_module(
    device: Union[str, Devices] = Devices.CPU
) -> Union[ModuleType, None]:
    if isinstance(device, str):
        device = device.lower()

    if device == Devices.CPU or device == Devices.CPU.value:
        import modules_lesson.computitions.cpu as module
        print('[INFO]: CPU computitions module was imported')
        return module

    if device == Devices.GPU or device == Devices.GPU.value:
        import modules_lesson.computitions.gpu as module
        print('[INFO]: GPU computitions module was imported')
        return module
    
    raise ValueError(f'invalid device value: {device}')

In [3]:
comp_cpu = load_computition_module('CPU')
comp_gpu = load_computition_module(Devices.GPU)

[INFO]: CPU computitions module was imported
[INFO]: GPU computitions module was imported


In [4]:
comp_cpu.compute()
comp_gpu.compute()

pass

Computing: \;
Computing: \;


Модуль может являться элементом некоторой коллекции или атрибутам класса. Также модуль является хэшируемым объектом, т.е. модуль можно хранить в качестве элемента в словаре и множестве:

In [6]:
computitions_modules = {
    device.value: load_computition_module(device) 
    for device in Devices
}

[INFO]: CPU computitions module was imported
[INFO]: GPU computitions module was imported


In [11]:
result = computitions_modules[Devices.GPU.value].compute()
print(f'{result = };'.capitalize())

Computing: \;
Result = 42;


## Ключевое слово import

Вы можете использовать любой исходный Python-файл в качеству модуля путем выполнения утверждения **import** в нужном вам исходном файле. Конструкция **import** имеет следующий синтаксис:

```python
import module_name [as name][, ...]
```

Рассмотрим эту конструкцию подробнее. После ключевого слова **import** всегда следует спецификатор модуля `module_name` или несколько спецификаторов модулей, разделенных запятыми. Спецификатор модуля указывает, какой именно модуль должен быть использован в текущем модуле. В простейшем, наиболее частом случае `module_name` один и выступает в роли переменной, с которой будет связан объект-модуль после завершения операции импорта. В данном случае Python будет искать файл `module_name.py`, чтобы удовлетворить запросу на импорт. 

Рассмотрим пример:
```python
import module_name
```

В данном случае интерпретатор будет искать модуль module_name, полученный модуль будет связан с переменной module_name, определенной в текущей области видимости и в текущем пространстве имен.

Помимо сказанного, module_name также может быть последовательностью идентефикаторов, разделенных точками, что позволяет осуществлять доступ к модулям из пакетов. Пример подобных импортов можно увидеть в листингах кода, приведенных выше.

Также конструкция import включает в себя необязательную часть `as name`. В этом случае интерпретатор будет искать модуль module_name, и полученный модуль будет связан с переменной name.

```python
import module_name as name
```

## Тело модуля

Тело модуля представляет собой непустое множество утверждений, хранящихся в исходном Python-файле. В Python нет специального синтаксиса для индикации модуля, поэтому любой валидный файл с разрешением *.py* может считаться таковым. Тело модуля выполняется в момент импорта данного модуля в текущей программе.

*Пример - тело модуля [cpu.py](./modules_lesson/computitions/cpu.py)*:

```python
"""
CPU computitions mock
"""

from time import time, sleep
from itertools import cycle

from modules_lesson.computitions.constants import (
    WHEEL_SYMBOLS,
    PAUSE_TIME,
    RESULT_MOCK,
)


CPU_TIME = 10               # время вычисления на GPU


def compute() -> None:
    wheel = cycle(WHEEL_SYMBOLS)
    time_start = time()

    while time() - time_start < CPU_TIME:
        print(f'\rComputing: {next(wheel)};', end='')
        sleep(PAUSE_TIME)

    print('')

    return RESULT_MOCK
```

## Атрибуты модуля

В момент выполнения импорта происходит создание нового пространства имен, которое содержит все атрибуты модуля. Атрибутами модуля называются объекты, определенные в глобальной области видимости файла. Для доступа к атрибутам, используется тот же синтаксис, что мы видели при доступе к атрибутам класса: указывается имя переменной, связанной с модулем, а через точку указывается имя интересующего нас атрибута.

In [13]:
import modules_lesson.computitions.gpu

modules_lesson.computitions.gpu.compute()
pass

Computing: \;


In [1]:
import modules_lesson.computitions.gpu as gpu

gpu.compute()
pass

Computing: \;


Обычное назначение модулей - создавать и хранить атрибуты, для упорядочивания кода. В теле функции обычно определяются функции, пользовательские классы или глобальные переменные (проектные константы). Обычно в рамках одного модуля реализуются объекты тесно связанные друг с другом и решающие одну конкретную бизнес-задачу. Если данный модуль не планируется использоваться в качестве исполняемого скрипта, избегайте использование каких-либо операций в теле этого модуля, помимо определения атрибутов.

По аналогии с классами, вы можете добавлять атрибуты в модуль из вне в процессе выполнения программы. Однако данная практика нежелательно, поскольку снижает выразительность и понятность вашей программы.

In [16]:
gpu.GPU = '1080 TI'
print(gpu.GPU)

1080 TI


При выполнении импорта, интерпретатор определяет некоторые атрибуты модуля неявным образом, как только создается объект модуля. К числу таких атрибутов относится атрибут \_\_dict\_\_ - объект, напоминающий словарь, скоторым мы встречались и раньше. В этом атрибуте осуществляется хранение пространства имен модуля. Атрибут \_\_dict\_\_, в отличие от других атрибутов модуля, типа \_\_name\_\_ недоступен в теле модуля. Еще одним атрибутом, задающимся неявным образом, является атрибут \_\_file\_\_ - путь к файлу, в котором хранится модуль:

In [19]:
print(gpu.__name__)
print(gpu.__file__)

modules_lesson.computitions.gpu
c:\Users\Michail\Desktop\mipt\teaching\python_mipt_dafe\lessons\lesson12\modules_lesson\computitions\gpu.py


Попытка доступа к неопределенному атрибуту модуля завершится ошибкой AttributeError:

In [20]:
gpu.compute_fast()

AttributeError: module 'modules_lesson.computitions.gpu' has no attribute 'compute_fast'

## Модули и встроенные функции

Как мы знаем, Python содержит большое количество встроенных объектов: функций и классов. Все эти объекты являются атрибутами предварительно загружаемого модуля builtins. Когда Python импортирует пользовательский модуль, интерпретатор автоматически заполняет атрибут \_\_builtins\_\_ данного модуля. Обозначенный атрибут или ссылается на модуль builtins или ссылается на его пространство имен, т.е. на builtins.\_\_dict\_\_.

Если вы обращаетесь к какомо-то объекто по идентефикатору, который ни находится ни в одной из областей видимости данного модуля, Python не возбуждает исключение мгновенно. Вместо этого он осуществляет поиск нужного имени в трибуте \_\_builtins\_\_.

## Приватные атрибуты модуля

Как и в случае с классами, ни один атрибут модуля не является истинно приватным. Однако, согласно известному соглашению, все объекты, чьи идентефикаторы начинаются с нижнего подчеркивания, считаются приватными. Старайтесь избегать прямого использование приватных атрибутов модулей в своем коде, особенно в случаях со сторонними библиотеками. Ведь, в большенстве случаем разработчики библиотеки гарантируют поддержку интерфейса или сообщают о его устаревании заранее, что не справедливо для реализации. Так, множество приватных атрибутов модулей библиотеки может меняться от релиза к релизу.

## Ключевое слово from

Специальная конструкция **from** позволяет импортировать отдельные атрибуты модуля в текущее пространство имен. Конструкция from имеет две формы:

```python
from module_name import attrname [as name][, ...]   # первая форма
from module_name import *                           # вторая форма
```

### Первая форма

Как и прежде, `module_name` является спецификатором модуля, на нем мы останавливаться не будем. `attrname` - идентефикатор атрибута модуля module_name, или идентефикаторы атрибутов данного модуля, разделенные запятыми. В простейшем случае, attrname - единственны идентефикатор:

```Python
from module_name import attrname
```

В этом случае интерпретатор загружает атрибут attrname модуля module_name и связывает его с переменной attrname текущего простраства имен. Если с помощью конструкции `as name` явным образом указан идентефикатор импортируемого аргумента, attrname будет связан с переменной name текущего пространства имен:

```Python
from module_name import attrname as name
```

Вы можете импортировать сразу несколько атрибутов из одного модуля в рамках одного выполнения конструкции from. Если число импортируемых атрибутов столь велико, что не может быть корректно размещено в рамках одной физической строки, вы можете представить это физическую строку в виде многострочной логический строки путем использования круглых скобок:

```Python
from module_name import (
    attrname1 as name1,
    attrname2,
    attrname3,
    attrname4 as name4,
)
```

### Вторая форма

Данная форма может быть использована только на уровне модуля. В данном контексте звездочка обозначает импорт всех публичных атрибутов модуля в текущее пространство имен. Если в модуле, из которого осуществляется импорт, определен специальный атрибут \_\_all\_\_, равный списку атрибутов этого модуля, при выполнении данной формы импорта в текущее пространство имен будут перенесены значения из \_\_all\_\_.

Опасайтесь использования этой форма импорта, поскольку импортированные значения могут перекрыть собой существующие переменные. Или наоборот, определяемые в дальнейшем переменные перекроют собой импортированные переменные, что приведет к путанице и дальнейшим сложностям с отладкой программы. 